In [5]:
import itertools as it
from collections import Counter, defaultdict
import math
import numpy as np

In [6]:
# -----------------------------
# Utilities: partitions & hooks
# -----------------------------

def partitions_of_n(n):
    """Generate all integer partitions of n in nonincreasing order."""
    def gen(n, max_part):
        if n == 0:
            yield []
            return
        for x in range(min(n, max_part), 0, -1):
            for rest in gen(n-x, x):
                yield [x] + rest
    return [tuple(p) for p in gen(n, n)]

def dim_specht(n, i):
    # valid for 0 <= i <= n
    # handles i=0 via the binomial identity with C(n,-1)=0
    return math.comb(n, i) - (math.comb(n, i-1) if i > 0 else 0)

# -----------------------------
# Permutations & cycle types
# -----------------------------

def perm_compose(p, q):
    """Compose permutations p∘q as tuples/lists of 0..n-1 (apply q then p)."""
    return tuple(p[i] for i in q)

def perm_inverse(p):
    n = len(p)
    inv = [0]*n
    for i, pi in enumerate(p):
        inv[pi] = i
    return tuple(inv)

def perm_identity(n): return tuple(range(n))

def all_perms(n):
    """All permutations of size n (WARNING: n! growth)."""
    return list(it.permutations(range(n)))

def cycle_type(p):
    """Return cycle type of permutation p as a partition (lengths in nonincreasing order)."""
    n = len(p)
    seen = [False]*n
    cyc = []
    for i in range(n):
        if not seen[i]:
            j=i; L=0
            while not seen[j]:
                seen[j] = True
                j = p[j]
                L += 1
            cyc.append(L)
    cyc.sort(reverse=True)
    return tuple(cyc)

def canonical_rep_from_cycle_type(n, part):
    """
    Construct a simple representative permutation of cycle type 'part'
    on {0,1,...,n-1}.
    """
    p = list(range(n))
    idx = 0
    for L in part:
        # cycle (idx idx+1 ... idx+L-1)
        for j in range(L-1):
            p[idx + j] = idx + j + 1
        p[idx + L - 1] = idx
        idx += L
    return tuple(p)

def conjugacy_classes_sn(n):
    """
    Return list of dicts: {'type':part, 'size':|C|, 'rep':representative permutation}
    """
    classes = []
    for part in partitions_of_n(n):
        # Class size formula: n! / ( Π_i (m_i! * i^{m_i}) )
        # where m_i = multiplicity of i in partition
        mult = Counter(part)
        denom = 1
        for i, m in mult.items():
            denom *= (math.factorial(m) * (i**m))
        size = math.factorial(n) // denom
        rep = canonical_rep_from_cycle_type(n, part)
        classes.append({'type': part, 'size': size, 'rep': rep})
    return classes


In [7]:

# ---------------------------------------
# Representation on k-subsets: ρ(σ)
# ---------------------------------------

def k_subsets(n, k):
    return [tuple(c) for c in it.combinations(range(n), k)]

def action_on_k_subsets_matrix(n, k, sigma):
    """
    Build the permutation matrix ρ(sigma) on the basis {e_A}_{A in X_k}.
    """
    Xk = k_subsets(n, k)
    idx = {A:i for i,A in enumerate(Xk)}
    m = len(Xk)
    M = np.zeros((m, m), dtype=float)
    for j, A in enumerate(Xk):
        # apply sigma to A
        B = tuple(sorted(sigma[a] for a in A))
        i = idx[B]
        M[i, j] = 1.0
    return M

# ---------------------------------------
# Characters via Murnaghan–Nakayama
# ---------------------------------------

def rim_hooks(partition, length):
    """
    Enumerate all rim hooks of the Young diagram of 'partition' having total length 'length'.
    A rim hook is a connected skew border strip with no 2x2 block.
    Return list of tuples (new_partition, height) where height = (#rows spanned - 1).
    """
    lam = list(partition)
    r = len(lam)

    # Represent diagram as set of boxes (i,j) with i=0..r-1, j=0..lam[i]-1
    boxes = {(i,j) for i in range(r) for j in range(lam[i])}

    # Find all border strips by DFS that start from outer rim boxes
    # Strategy: start from any outer boundary box, try to walk along rim removing 'length' boxes.
    # We'll approximate: try all connected paths of length 'length' along rim that never include 2x2.
    # First, find outer rim cells: a box is on rim if (i,j+1) not in boxes or (i-1,j) not in boxes.
    rim_cells = set()
    for (i,j) in boxes:
        if ((i, j+1) not in boxes) or ((i-1, j) not in boxes):
            rim_cells.add((i,j))

    # adjacency on rim: neighbors (i,j) -> {(i, j-1),(i, j+1),(i+1, j),(i-1, j)} intersected with rim
    nbrs = lambda b: [c for c in [(b[0],b[1]-1),(b[0],b[1]+1),(b[0]+1,b[1]),(b[0]-1,b[1])]
                      if c in rim_cells]

    # helper to check no 2x2 block in a set of boxes S
    def has_2x2(S):
        Sset = set(S)
        for (i,j) in Sset:
            if (i+1,j) in Sset and (i,j+1) in Sset and (i+1,j+1) in Sset:
                return True
        return False

    hooks = set()
    # Try all starting rim cells; do simple DFS paths of exact length
    for start in rim_cells:
        stack = [(start, [start])]
        visited_local = set()
        while stack:
            node, path = stack.pop()
            if len(path) == length:
                # border strip candidate
                S = path
                if not has_2x2(S):
                    rows_spanned = {i for (i,_) in S}
                    height = len(rows_spanned) - 1
                    # Remove S from diagram -> new partition
                    B = boxes - set(S)
                    # Clean trailing zeros per row
                    new_row_lengths = []
                    for i in range(r):
                        rowlen = sum((i,j) in B for j in range(lam[i]))
                        if rowlen>0:
                            new_row_lengths.append(rowlen)
                    new_part = tuple(sorted(new_row_lengths, reverse=True))
                    hooks.add((new_part, height))
                continue
            # extend along rim
            for nb in nbrs(node):
                if nb not in path:
                    stack.append((nb, path+[nb]))

    return list(hooks)

from functools import lru_cache

@lru_cache(None)
def mn_character(lambda_part, cycle_type_part):
    """
    Murnaghan–Nakayama rule:
    χ^λ(μ) = sum_{rim hooks of lengths μ1,μ2,...} (-1)^{height sum}
    Implemented recursively: remove one cycle length each time.
    """
    lam = tuple(lambda_part)
    mu = tuple(cycle_type_part)

    if sum(lam) != sum(mu):
        return 0
    if sum(mu) == 0:
        return 1 if sum(lam)==0 else 0
    # choose one cycle length m from mu (use first)
    m = mu[0]
    rest = mu[1:]

    total = 0
    # enumerate rim hooks of size m
    for new_part, height in rim_hooks(lam, m):
        s = (-1)**height * mn_character(new_part, rest)
        total += s
    return total

def irreducible_character_two_row(n, i, cycle_type_part):
    """Convenience wrapper: λ=(n-i,i)."""
    lam = (n - i, i) if i <= n - i else tuple(sorted((n-i, i), reverse=True))
    return mn_character(lam, cycle_type_part)

# ---------------------------------------
# Central idempotent projectors on R[X_k]
# ---------------------------------------

def projectors_two_row_vnk(n, k):
    """
    Build p_{(n-i,i)} on V_{n,k} for i=0..k using central idempotent sum over classes.
    Returns:
      - proj dict: i -> p_lambda (matrix)
      - Q dict: i -> orthonormal basis for im(p_lambda) (columns)
    """
    # basis & index
    basis = k_subsets(n, k)
    m = len(basis)

    # conjugacy classes
    classes = conjugacy_classes_sn(n)

    # Precompute ρ(gC) for each class rep (these are permutation matrices)
    rho_class = {}
    for C in classes:
        rho_class[C['type']] = action_on_k_subsets_matrix(n, k, C['rep'])

    # Build each projector
    proj = {}
    Qblocks = {}
    for i in range(0, k+1):
        lam = (n-i, i) if i <= n-i else tuple(sorted((n-i, i), reverse=True))
        dimL = dim_specht(n, i) 
        # class-sum
        M = np.zeros((m, m), dtype=float)
        for C in classes:
            mu = C['type']
            chi = irreducible_character_two_row(n, i, mu)
            M += C['size'] * chi * rho_class[mu]
        p = (dimL / math.factorial(n)) * M
        # symmetrize numerically
        p = 0.5*(p + p.T)
        proj[i] = p

        # extract orthonormal basis for image: eigenvectors with eigenvalue ~1
        w, V = np.linalg.eigh(p)  # symmetric
        # take those closest to 1
        dim_block = dimL
        idx = np.argsort(np.abs(w - 1.0))[:dim_block]
        Qi = V[:, idx]
        # re-orthonormalize (Gram-Schmidt) for safety
        Qi, _ = np.linalg.qr(Qi)
        Qblocks[i] = Qi

    return proj, Qblocks, basis, classes

# ---------------------------------------
# Assemble global Q and verify
# ---------------------------------------

def assemble_Q(Qblocks, order=None):
    if order is None:
        order = sorted(Qblocks.keys())
    return np.concatenate([Qblocks[i] for i in order], axis=1)

def verify_projectors(proj, dims, tol=1e-8):
    keys = sorted(proj.keys())
    ok = True
    # idempotence and ranks
    for i in keys:
        p = proj[i]
        if np.linalg.norm(p@p - p) > 1e-6:
            print(f"[warn] projector {i} not idempotent within tolerance")
            ok = False
        tr = np.trace(p)
        if abs(tr - dims[i]) > 1e-5:
            print(f"[warn] trace(p_{i})={tr} differs from dim {dims[i]}")
            ok = False
    # orthogonality and partition of identity
    S = np.zeros_like(proj[keys[0]])
    for i in keys:
        S += proj[i]
    if np.linalg.norm(S - np.eye(S.shape[0])) > 1e-5:
        print("[warn] sum of projectors not identity within tolerance")
        ok = False
    for i in keys:
        for j in keys:
            if i<j:
                if np.linalg.norm(proj[i]@proj[j]) > 1e-6:
                    print(f"[warn] projectors {i},{j} not orthogonal enough")
                    ok = False
    return ok

def block_diagonalize(n, k, Q, sigma):
    """Return block-diagonalised matrix Q^T rho(sigma) Q."""
    R = action_on_k_subsets_matrix(n, k, sigma)
    return Q.T @ R @ Q

In [8]:
# Example: n=6, k=2  (dims: C(6,2)=15; blocks for i=0,1,2)
n, k = 6, 2
proj, Qblocks, basis, classes = projectors_two_row_vnk(n, k)

# Expected dimensions for two-row (n-i,i)
dims = {i: (math.comb(n, i) - (math.comb(n, i-1) if i>0 else 0)) for i in range(0, k+1)}
print("Block dimensions:", dims)

ok = verify_projectors(proj, dims)
print("Verification passed:", ok)

# Assemble Q
Q = assemble_Q(Qblocks)
print("Q orthonormal check:", np.allclose(Q.T @ Q, np.eye(Q.shape[1]), atol=1e-6))

# Check block diagonalization for a generator (e.g. adjacent transposition (0 1))
sigma = list(range(n)); sigma[0], sigma[1] = sigma[1], sigma[0]; sigma = tuple(sigma)
N = block_diagonalize(n, k, Q, sigma)
# Show that off-block norms are small by peeking at block sizes
block_sizes = [dims[i] for i in range(0, k+1)]
cuts = np.cumsum([0] + block_sizes)
off_norm = 0.0
for a in range(len(block_sizes)):
    for b in range(len(block_sizes)):
        if a == b: continue
        A = N[cuts[a]:cuts[a+1], cuts[b]:cuts[b+1]]
        off_norm += np.linalg.norm(A)
print("Off-block Frobenius norm (should be ~0):", off_norm)

Block dimensions: {0: 1, 1: 5, 2: 9}
[warn] projector 0 not idempotent within tolerance
[warn] projector 1 not idempotent within tolerance
[warn] trace(p_1)=-1.1458333333333333 differs from dim 5
[warn] projector 2 not idempotent within tolerance
[warn] trace(p_2)=4.875 differs from dim 9
[warn] sum of projectors not identity within tolerance
[warn] projectors 0,1 not orthogonal enough
[warn] projectors 0,2 not orthogonal enough
[warn] projectors 1,2 not orthogonal enough
Verification passed: False
Q orthonormal check: False
Off-block Frobenius norm (should be ~0): 4.8676931583273815
